In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
from tqdm.notebook import tqdm


# Prepare data

## Load dataset

In [2]:
import csv
import os
import sys
import codecs
from glob import glob
from xml.dom.minidom import parse, parseString
from nltk import WordPunctTokenizer

print("entered xmlparser.py")

# Parcourir tout sous-dossiers et avoir les fichiers dans ces sous-dossiers.
def get_file_names(dirname):
    files = []
    os.chdir(dirname)
    start_dir = os.getcwd()
    pattern = "*.xml"
    for dir,_,_ in tqdm(os.walk(start_dir)):
        files = files + glob(os.path.join(dir, pattern))

    non_empty_files = []
    for fname in files:
        with codecs.open(fname, "r", encoding="utf-8") as f:
            ct = f.read()
            if len(ct) > 0:
                non_empty_files.append(fname)

    print("obtained " + str(len(non_empty_files)) + " files from " + start_dir) 
    print("returning to initial directory...")
    os.chdir(start_dir)
    os.chdir("..")
    print("current directory: " + os.getcwd())
    return non_empty_files



entered xmlparser.py


In [3]:
import csv
# un fichier = une jurisprudence
# Extraire les informations de chaque fichier(de chaque jurisprudence)
def parse_french_data(non_empty_files):
    count = 0
    L=[['idx', 'formation','solution','date','content','articles','citation']]
    for fname in tqdm(non_empty_files):
    #for fname in glob("*.xml"):
        try:
            dom=parse(fname)
            for node in dom.getElementsByTagName('ID'):
                idx = node.toxml()
            for node in dom.getElementsByTagName('BLOC_TEXTUEL'):
                contenu = node.toxml()
            for node in dom.getElementsByTagName('FORMATION'):
                formation = node.toxml()
            for node in dom.getElementsByTagName('DATE_DEC'):
                date = node.toxml()
            for node in dom.getElementsByTagName('DATE_DEC_ATT'):
                date_att = node.toxml()
            for node in dom.getElementsByTagName('SOLUTION'):
                solution=node.toxml()
            for node in dom.getElementsByTagName('LIEN'):
                citation = node.toxml()

            if ("<CONTENU/>" in contenu) and ("<CONTENU>" not in contenu):
                contenu = ""
            else:
                contenu=contenu.replace("<BLOC_TEXTUEL>","")
                contenu=contenu.replace("<CONTENU>", "")
                contenu=contenu.replace("</BLOC_TEXTUEL>","")
                contenu=contenu.replace("</CONTENU>", "")
                contenu=contenu.replace("<br/>","")
                contenu=contenu.replace("<p>","")
                contenu=contenu.replace("</p>","")
                contenu=contenu.replace("\n"," ")
                contenu=contenu.replace("\t"," ")


            if contenu == "":
                for node in dom.getElementsByTagName('SOMAIRE'):
                    contenu = node.toxml()

            content = WordPunctTokenizer().tokenize(contenu)
            articles = []
            
            for i in range(len(content)-2):
                if content[i].lower() == "par" and content[i+1].lower() == "ces" and content[i+2].lower() == "motifs" :
                    content=content[0:i]

            
                    
            

            for i in range(len(content)):
                if (content[i].lower() == "article") or (content[i] == "Article"):
                    articles.append(content[i+1])


            articles = " ".join(articles)
            content = ' '.join(content) # joins the elements of the list content

            idx = idx.replace("<ID>", "")
            idx = idx.replace("</ID>", "")

            if "<FORMATION/>" == formation:
                formation =""
            else:
                formation = formation.replace("<FORMATION>","")
                formation = formation.replace("</FORMATION>","")
                formation = formation.replace("\t"," ")
                formation = formation.replace("\n"," ")

            if "<DATE_DEC/>" == date:
                date =""
            else:
                date = date.replace("<DATE_DEC>","")
                date = date.replace("</DATE_DEC>","")
                date = date.replace("\t"," ")
                date = date.replace("\n"," ")

            if "<DATE_DEC_ATT/>" == date_att:
                date_att = ""
            else:
                date_att = date_att.replace("<DATE_DEC_ATT>","")
                date_att = date_att.replace("</DATE_DEC_ATT>","")
                date_att = date_att.replace("\t"," ")
                date_att = date_att.replace("\n"," ")

            if solution == "<SOLUTION/>": # if solution field is empty
                solution = "" # replace field with "" to simplify
            else: # else clean up
                solution = solution.replace("<SOLUTION>","")
                solution = solution.replace("</SOLUTION>","")
                solution = solution.replace(".","")
                solution = solution.replace("non lieu","non-lieu")
                solution = solution.replace("\t"," ")
                solution = solution.replace("\n"," ")

            if re.search(">.+<", citation) == None:
                citation = ""
            else:
                citation = (re.search(">.+<", citation)).group()
                citation = citation[1:len(citation)-1]
                citation = citation.replace("\t", " ")
                citation = citation.replace("\n", " ")

            if "2999" in date_att:
                csv = idx + "\t"+formation+"\t"+solution+"\t"+date+"\t"+content+"\t"+articles+"\t"+citation+"\n"
            else:
                csv = idx + "\t"+formation+"\t"+solution+"\t"+date_att+"\t"+content+"\t"+articles+"\t"+citation+"\n"
            
            count = count + 1
            # L[count][0]=idx
            # L[count][1]=formation
            # L[count][2]=
            # L[count][3]=
            # L[count][4]=
            # L[count][5]=
            L=L+[[str(idx), str(formation) , str(solution), str(date), str(content) , str(articles) , str(citation)]]        
        except Exception as er:
            print(er)

    
    return L

In [4]:
import csv
path = '/home/nono/Documents/Legal/Data/Freemium_legi_global_20220417-170000'  # DOSSIER qui contient l'ensemble des fichiers XML
files=get_file_names(path)

print("parsing")
M=parse_french_data(files)
len(M)
Headers=M.pop(0)
df = pd.DataFrame(M, columns=Headers)
df


0it [00:00, ?it/s]

KeyboardInterrupt: 

In [ ]:
df.to_csv(r'/home/nono/Documents/Legal/Data/Jurisprudences_juri_1.0.csv', index = False)


In [ ]:
len(df['content'][100])



2528

In [ ]:
from transformers import CamembertTokenizer, CamembertModel
model_name = "camembert-base"
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertModel.from_pretrained('camembert-base')

In [ ]:
# convert les phrases en miniscule et tokenisation avec Camemebert tokenizer
corpus_tokens_ = [tokenizer.tokenize(doc.lower()) for doc in df.content]
assert(len(corpus_tokens_)==len(df))
corpus_tokens_[0] #la première phrase tokenized
df['tokenized_sent'] = corpus_tokens_
df.head()
df.tokenized_sent[0]

['▁rejet',
 '▁du',
 '▁pour',
 'voi',
 '▁formé',
 '▁par',
 '▁zo',
 'é',
 '▁-',
 '▁marie',
 '▁-',
 '▁l',
 'ou',
 'ise',
 '▁her',
 'bin',
 '▁contre',
 '▁un',
 '▁arrêt',
 '▁rendu',
 '▁par',
 '▁la',
 '▁cour',
 '▁impériale',
 '▁d',
 '▁',
 "'",
 '▁or',
 'lé',
 'ans',
 '▁',
 ',',
 '▁le',
 '▁6',
 '▁juillet',
 '▁18',
 '61',
 '▁',
 ',',
 '▁en',
 '▁faveur',
 '▁du',
 '▁si',
 'eur',
 '▁x',
 '▁...',
 ',',
 '▁son',
 '▁mari',
 '▁',
 '.',
 '▁du',
 '▁24',
 '▁avril',
 '▁18',
 '62',
 '▁',
 '.',
 '▁la',
 '▁cour',
 '▁',
 ',',
 '▁chambres',
 '▁réunies',
 '▁',
 ',',
 '▁ou',
 'ï',
 '▁m',
 '▁',
 '.',
 '▁le',
 'g',
 'agne',
 'ur',
 '▁',
 ',',
 '▁conseiller',
 '▁',
 ',',
 '▁en',
 '▁son',
 '▁rapport',
 '▁;',
 '▁maître',
 '▁am',
 'br',
 'oise',
 '▁a',
 '▁...',
 ',',
 '▁en',
 '▁ses',
 '▁observations',
 '▁',
 ',',
 '▁à',
 '▁l',
 '▁',
 "'",
 '▁audience',
 '▁publique',
 '▁du',
 '▁22',
 '▁avril',
 '▁;',
 '▁maître',
 '▁z',
 '▁...',
 ',',
 '▁en',
 '▁ses',
 '▁observations',
 '▁',
 ',',
 '▁et',
 '▁m',
 '▁',
 '.',
 '▁le',
 '▁

In [ ]:
tokenizer.encode(df.tokenized_sent[0]) 

Token indices sequence length is longer than the specified maximum sequence length for this model (723 > 512). Running this sequence through the model will result in indexing errors


[5,
 6742,
 25,
 24,
 8244,
 6234,
 37,
 11508,
 141,
 67,
 9958,
 67,
 17,
 308,
 1914,
 7100,
 7368,
 192,
 23,
 4423,
 1736,
 37,
 13,
 1966,
 20403,
 18,
 21,
 11,
 1632,
 1357,
 2160,
 21,
 7,
 16,
 260,
 742,
 301,
 5648,
 21,
 7,
 22,
 2558,
 25,
 86,
 601,
 758,
 186,
 7,
 58,
 1946,
 21,
 9,
 25,
 641,
 871,
 301,
 4959,
 21,
 9,
 13,
 1966,
 21,
 7,
 1310,
 17789,
 21,
 7,
 47,
 4525,
 115,
 21,
 9,
 16,
 383,
 4483,
 297,
 21,
 7,
 2256,
 21,
 7,
 22,
 58,
 459,
 167,
 2300,
 4098,
 3239,
 2885,
 33,
 186,
 7,
 22,
 89,
 10126,
 21,
 7,
 15,
 17,
 21,
 11,
 10238,
 1275,
 25,
 837,
 871,
 167,
 2300,
 4737,
 186,
 7,
 22,
 89,
 10126,
 21,
 7,
 14,
 115,
 21,
 9,
 16,
 10266,
 606,
 25,
 3424,
 21,
 7,
 22,
 89,
 9351,
 21,
 7,
 15,
 17,
 21,
 11,
 10238,
 1275,
 18,
 21,
 11,
 2067,
 167,
 14531,
 172,
 16,
 26312,
 22,
 608,
 25,
 993,
 167,
 4903,
 27,
 17,
 21,
 11,
 3343,
 29,
 13,
 314,
 174,
 19,
 1213,
 476,
 752,
 14,
 6326,
 25,
 102,
 186,
 49,
 1759,
 7115,
 88,
